In [2]:
import os
os.chdir("D:/Flood_Project/Model_running/Trail Run-3/Network_file_preprocessing")

In [3]:
import pandas as pd
print(pd.__version__)

2.2.2


In [8]:
df_csv = pd.read_csv('SEMCOG_Risk_Road.csv',   low_memory=False)

In [9]:
df_csv.head(2)

,RoadSegmen,PR,Linearef,FETYPE,FENAME,BMP,EMP,all_roads_,spatialjoi,spatialj_1,...,all_roads1,perrank_nf,perrank_cr,perrank_vu,perrank_ri,tampnum_ne,ramp,completed_,risk_level,shape_Leng
0,278779,964704,9647047400002074000008,Rd,Lakeshore Rd,12.517,12.598,147,Lakeshore Rd,Huron Walk,...,3,3.0,2.666667,1.465,1.765417,0,n,NaN,Low,426.267071
1,279137,964704,9647047400002874000020,Rd,Lakeshore Rd,12.412,12.517,147,Lakeshore Rd,Huron Walk,...,3,3.0,2.666667,1.465,1.765417,0,n,NaN,Low,555.969541


In [7]:
import geopandas as gpd

In [10]:
df_shp = gpd.read_file('detroit_road.shp')

In [18]:
import pandas as pd

# Assuming `df_csv` is the DataFrame from the CSV file and `df_shp` is the DataFrame from the DBF of the shapefile

# Step 1: Directly merge on unique PR values
unique_pr_csv = df_csv[~df_csv['PR'].duplicated(keep=False)]
unique_pr_shp = df_shp[~df_shp['PR'].duplicated(keep=False)]
merged_unique_pr = pd.merge(unique_pr_shp, unique_pr_csv, on=['PR', 'BMP', 'EMP'], how='inner')

# Step 2: Handle duplicate PR values
dup_pr_csv = df_csv[df_csv['PR'].duplicated(keep=False)]
dup_pr_shp = df_shp[df_shp['PR'].duplicated(keep=False)]

# Initialize an empty list to store the merged rows for duplicates
merged_dup_pr_rows = []

# Loop over each row in `dup_pr_shp` (only rows in shapefile should be retained in the final result)
for _, row_shp in dup_pr_shp.iterrows():
    # Filter rows in `dup_pr_csv` with the same `PR`
    matching_csv_rows = dup_pr_csv[dup_pr_csv['PR'] == row_shp['PR']]
    
    if not matching_csv_rows.empty:
        # Step 2.1: Check for exact matches on BMP
        bmp_match = matching_csv_rows[matching_csv_rows['BMP'] == row_shp['BMP']]
        
        if not bmp_match.empty:
            # If a BMP match is found, add to the result
            merged_dup_pr_rows.append(pd.concat([row_shp, bmp_match.iloc[0]], axis=1))
        else:
            # Step 2.2: If no BMP match, check for exact match on EMP
            emp_match = matching_csv_rows[matching_csv_rows['EMP'] == row_shp['EMP']]
            
            if not emp_match.empty:
                # If an EMP match is found, add to the result
                merged_dup_pr_rows.append(pd.concat([row_shp, emp_match.iloc[0]], axis=1))
            else:
                # Step 2.3: If no match on BMP or EMP, find the closest BMP
                matching_csv_rows['bmp_difference'] = (matching_csv_rows['BMP'] - row_shp['BMP']).abs()
                closest_bmp_match = matching_csv_rows.loc[matching_csv_rows['bmp_difference'].idxmin()]
                merged_dup_pr_rows.append(pd.concat([row_shp, closest_bmp_match], axis=1))

# Concatenate all merged rows for duplicate PR values
if merged_dup_pr_rows:
    merged_dup_pr = pd.concat(merged_dup_pr_rows, axis=1).T
else:
    merged_dup_pr = pd.DataFrame()

# Step 3: Combine the unique and duplicate PR merges, ensuring only rows from the shapefile remain
final_merged_df = pd.concat([merged_unique_pr, merged_dup_pr], ignore_index=True)

# Drop any rows that were not originally in the shapefile data (keeping only PRs from `df_shp`)
final_merged_df = final_merged_df[final_merged_df['PR'].isin(df_shp['PR'].unique())]


<ipython-input-18-f9363c0cadc6>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_csv_rows['bmp_difference'] = (matching_csv_rows['BMP'] - row_shp['BMP']).abs()
<ipython-input-18-f9363c0cadc6>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_csv_rows['bmp_difference'] = (matching_csv_rows['BMP'] - row_shp['BMP']).abs()
<ipython-input-18-f9363c0cadc6>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [19]:
final_merged_df

,ID,DIR,LENGTH,AB_LANES,BA_LANES,CENT_LANE,MODE_ID,NFC,NFC_FLAG,AREA_TYPE,...,perrank_vu,perrank_ri,tampnum_ne,ramp,completed_,risk_level,shape_Leng,FENAME,FETYPE,bmp_difference
0,39957,1,0.16,3,0,0,2,5,None,3,...,2.86,2.561667,0,n,NaN,Medium,819.07689,NaN,NaN,NaN
1,39932,1,0.16,2,0,0,1,3,None,3,...,2.365,2.690417,0,n,NaN,Medium,860.148025,NaN,NaN,NaN
2,39933,1,0.69,2,0,0,1,3,None,3,...,1.99,2.2425,0,n,NaN,Medium,3650.074905,NaN,NaN,NaN
3,22545,-1,0.15,0,2,0,2,4,None,3,...,1.81,2.1075,0,n,NaN,Medium,767.751115,NaN,NaN,NaN
4,15899,1,0.21,1,0,0,1,1,RON,3,...,2.275,2.372917,0,y,NaN,Medium,1118.312122,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.1975,3.148125,0,n,NaN,High,326.163313,Livernois Ave,Ave,NaN
11275,39855,0,0.01,2,2,0,2,4,None,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bagley,St,NaN
11276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.29,2.300833,0,n,NaN,Medium,42.549792,Bagley St,St,NaN
11277,39857,-1,0.06,0,2,0,2,4,DV2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3rd,St,NaN


In [22]:
final_merged_df.to_csv('Balsal.csv')

In [23]:
import geopandas as gpd
from shapely.geometry import LineString

In [25]:
# Convert incompatible types to standard Python types
for column in final_merged_df.columns:
    if final_merged_df[column].dtype == 'int64':
        final_merged_df[column] = final_merged_df[column].astype(int)
    elif final_merged_df[column].dtype == 'float64':
        final_merged_df[column] = final_merged_df[column].astype(float)
    elif final_merged_df[column].dtype == 'object':
        final_merged_df[column] = final_merged_df[column].astype(str)  # Ensure all object types are strings

# Now create the GeoDataFrame and export
import geopandas as gpd
from shapely.geometry import LineString

# Assuming 'geometry' column is already present with LineString geometries
gdf = gpd.GeoDataFrame(final_merged_df, geometry='geometry')
gdf.set_crs("EPSG:4326", inplace=True)  # Set the CRS

# Export to shapefile
gdf.to_file("final_merged.shp")

<ipython-input-25-2a92aac5cc25>:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("final_merged.shp")
